# Part I: Bloqade pipeline — Squin kernels only

**QuEra iQuHack 2026 — Step 1**

All circuits are defined as **Squin kernels** (Bloqade's dialect). We emit them to Cirq with **emit_circuit**, then convert back to Squin with **load_circuit** for Stim/Tsim simulation.

---

### Running this notebook

| Where | What to do |
|-------|------------|
| **Google Colab** | Run the **Install dependencies** cell first. Then **Runtime → Restart session**. Run the rest in order. |
| **Local** | Use the project's `.venv` kernel, or run the install cell, restart kernel, then run all. |

## 1. Install dependencies

We need: **numpy**, **cirq**, and **bloqade-circuit** (with cirq, stim, tsim). Run once.

**Colab:** After it finishes, use **Runtime → Restart session**, then run the rest.

In [ ]:
!pip install -q numpy cirq "bloqade-circuit[cirq,stim,tsim]>=0.11.0"
print("Done. On Colab: Runtime -> Restart session, then run the rest.")

## 2. Imports

- **bloqade.squin**: define kernels with **@squin.kernel**, **squin.qalloc**, **squin.h**, **squin.cx**, **squin.depolarize**, **squin.depolarize2**, **squin.broadcast.measure**
- **emit_circuit**: convert a Squin kernel (uncalled) to a **Cirq** circuit
- **load_circuit**: convert a Cirq circuit back to **Squin** (so Stim/Tsim can run it)
- **bloqade.stim** / **bloqade.tsim**: run the Squin circuit and sample
- **noise**: apply QuEra's heuristic noise (e.g. **GeminiOneZoneNoiseModel**) to a Cirq circuit

In [ ]:
import numpy as np
from bloqade import squin
import bloqade.stim
import bloqade.tsim
from bloqade.cirq_utils import noise
from bloqade.cirq_utils import load_circuit
from bloqade.cirq_utils.emit import emit_circuit

## 3. Squin kernel: ideal 3-qubit GHZ

The **GHZ state** is $|\text{GHZ}\rangle = (|000\rangle + |111\rangle)/\sqrt{2}$.

We define it as a **Squin kernel**: **qalloc(3)** allocates 3 qubits, then **H** on q0, **CNOT** chain, then **measure** all. Pass the **kernel function** (do not call it) to **emit_circuit** later.

In [ ]:
@squin.kernel
def ghz_ideal_3():
    """Prepare (|000⟩ + |111⟩)/√2 and measure. All gates in Squin."""
    q = squin.qalloc(3)
    squin.h(q[0])
    squin.cx(q[0], q[1])
    squin.cx(q[1], q[2])
    squin.broadcast.measure(q)

## 4. Squin kernel: GHZ with manual depolarizing noise

Same GHZ, but we insert **squin.depolarize(p1, qubit)** (1-qubit) and **squin.depolarize2(p2, q0, q1)** (2-qubit) after gates. Parameters **p1** and **p2** are passed when we emit: **emit_circuit(ghz_manual_noise_3, args=(p1, p2))**.

In [ ]:
@squin.kernel
def ghz_manual_noise_3(p1: float, p2: float):
    """3-qubit GHZ with depolarize(p1) and depolarize2(p2) after gates."""
    q = squin.qalloc(3)
    squin.h(q[0])
    squin.depolarize(p1, q[0])
    squin.cx(q[0], q[1])
    squin.depolarize2(p2, q[0], q[1])
    squin.cx(q[1], q[2])
    squin.depolarize2(p2, q[1], q[2])
    squin.depolarize(p1, q[1])
    squin.depolarize(p1, q[2])
    squin.broadcast.measure(q)

## 5. Helpers: parity and report

**Parity OK** = fraction of shots where all qubits agree (all 0s or all 1s). Ideal GHZ gives **1.0**; noise lowers it.

In [ ]:
def ghz_parity_fraction(samples):
    if samples.size == 0:
        return 0.0
    all_zero = (samples == 0).all(axis=1)
    all_one = (samples == 1).all(axis=1)
    return np.logical_or(all_zero, all_one).mean()

def report(name, samples):
    parity = ghz_parity_fraction(samples)
    print(f"  {name}: shots={samples.shape[0]}, parity_ok={parity:.4f}, shape={samples.shape}")

## 6. Ideal simulation (no noise)

Pipeline: **Squin kernel → emit_circuit(kernel)** → Cirq → **load_circuit** → Squin → **Stim** or **Tsim** → sample.

We pass the **kernel function** (e.g. **ghz_ideal_3**) to **emit_circuit** without calling it. Expect **parity_ok ≈ 1.0**.

In [ ]:
shots = 2000

print("Part I: Bloqade pipeline (Squin kernels only)")
print("=" * 60)

print("\n--- 1. Ideal GHZ (Stim) ---")
cirq_circ = emit_circuit(ghz_ideal_3)
squin_circ = load_circuit(cirq_circ)
stim_circ = bloqade.stim.Circuit(squin_circ)
ideal_stim = np.array(stim_circ.compile_sampler().sample(shots=shots))
report("ideal_stim", ideal_stim)

print("\n--- 2. Ideal GHZ (Tsim) ---")
cirq_circ = emit_circuit(ghz_ideal_3)
squin_circ = load_circuit(cirq_circ)
tsim_circ = bloqade.tsim.Circuit(squin_circ)
ideal_tsim = np.array(tsim_circ.compile_sampler().sample(shots=shots))
report("ideal_tsim", ideal_tsim)

## 7. Manual noise: depolarizing channels

Emit **ghz_manual_noise_3** with **args=(0.02, 0.01)** so the kernel is instantiated with p1=0.02, p2=0.01. Then same pipeline: load_circuit → Stim → sample. **parity_ok** should drop (e.g. ~0.93–0.96).

In [ ]:
print("--- 3. Manual noise (p1=0.02, p2=0.01) ---")
cirq_circ = emit_circuit(ghz_manual_noise_3, args=(0.02, 0.01))
squin_circ = load_circuit(cirq_circ)
stim_circ = bloqade.stim.Circuit(squin_circ)
manual = np.array(stim_circ.compile_sampler().sample(shots=shots))
report("manual_noise", manual)

## 8. Heuristic hardware noise (GeminiOneZone)

Emit **ghz_ideal_3** to Cirq, then **noise.transform_circuit(cirq_circ, model=GeminiOneZoneNoiseModel(scaling_factor=0.1))**, then **load_circuit** → Stim. **parity_ok** is usually high when scale is small.

In [ ]:
print("--- 4. Heuristic noise (GeminiOneZone, scaling=0.1) ---")
cirq_circ = emit_circuit(ghz_ideal_3)
noise_model = noise.GeminiOneZoneNoiseModel(scaling_factor=0.1)
cirq_noisy = noise.transform_circuit(cirq_circ, model=noise_model)
squin_noisy = load_circuit(cirq_noisy)
stim_circ = bloqade.stim.Circuit(squin_noisy)
heuristic = np.array(stim_circ.compile_sampler().sample(shots=shots))
report("heuristic_noise", heuristic)
print("\nDone.")

## 9. Summary and Tsim diagram

| Run | Source | parity_ok (typical) |
|-----|--------|---------------------|
| Ideal (Stim/Tsim) | **ghz_ideal_3** kernel | 1.0 |
| Manual noise | **ghz_manual_noise_3** kernel | &lt; 1.0 |
| Heuristic | **ghz_ideal_3** → Cirq → noise | high (~0.99+) |

Below: print summary and show the **Tsim circuit diagram** for the ideal GHZ kernel.

In [ ]:
print("Summary:")
print(f"  Ideal (Stim):  parity_ok = {ghz_parity_fraction(ideal_stim):.4f}")
print(f"  Ideal (Tsim):  parity_ok = {ghz_parity_fraction(ideal_tsim):.4f}")
print(f"  Manual noise:  parity_ok = {ghz_parity_fraction(manual):.4f}")
print(f"  Heuristic:     parity_ok = {ghz_parity_fraction(heuristic):.4f}")

print("\nTsim circuit diagram (from ghz_ideal_3):")
cirq_diagram = emit_circuit(ghz_ideal_3)
squin_for_diagram = load_circuit(cirq_diagram)
tsim_circ = bloqade.tsim.Circuit(squin_for_diagram)
fig = tsim_circ.diagram(height=400)
fig